# ML Pipeline Preparation
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, inspect
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer,TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, accuracy_score,  precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
import warnings
import pickle
warnings.filterwarnings('ignore')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\20812015101195\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\20812015101195\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\20812015101195\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [3]:
db_file = 'data/DisasterResponse.db'

In [4]:
engine = create_engine(f'sqlite:///{db_file}')

In [5]:
# Get table information
inspector = inspect(engine)

# get table name
print(inspector.get_table_names())

['DisasterResponse']


In [6]:
# load DB file into data frame
df = pd.read_sql('select * from DisasterResponse',con=engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# load data from database
X = df['message']
Y = df.iloc[:,4:]

In [8]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [9]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [10]:
def tokenize(text):
    """
    convert text into clean tokens
    Args: text
    Return: list of tokens
    """
    # normalize text; remove all special charchters and convert all letters to lowercase 
    text = re.sub(r'[^a-zA-Z0-9]',' ',text.lower())
    
    # token messages
    words = word_tokenize(text)
    
    # removing stop words
    tokens = [w for w in words if w not in stopwords.words("english")]
    
    # emmtization
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

In [11]:
X[:5].apply(tokenize)

0    [weather, update, cold, front, cuba, could, pa...
1                                          [hurricane]
2                             [looking, someone, name]
3    [un, report, leogane, 80, 90, destroyed, hospi...
4    [say, west, side, haiti, rest, country, today,...
Name: message, dtype: object

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [12]:
pipeline = Pipeline([
    ('vect',TfidfVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=200)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [13]:
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.3, random_state=42)
pipeline.fit(X_train,y_train)

Pipeline(steps=[('vect',
                 TfidfVectorizer(tokenizer=<function tokenize at 0x000002AD4ECE4940>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=200)))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [14]:
classes = list(Y.columns)
classes

['related',
 'request',
 'offer',
 'aid_related',
 'medical_help',
 'medical_products',
 'search_and_rescue',
 'security',
 'military',
 'child_alone',
 'water',
 'food',
 'shelter',
 'clothing',
 'money',
 'missing_people',
 'refugees',
 'death',
 'other_aid',
 'infrastructure_related',
 'transport',
 'buildings',
 'electricity',
 'tools',
 'hospitals',
 'shops',
 'aid_centers',
 'other_infrastructure',
 'weather_related',
 'floods',
 'storm',
 'fire',
 'earthquake',
 'cold',
 'other_weather',
 'direct_report']

In [15]:
y_pred = pipeline.predict(X_test)

In [16]:
def display_results(y_test, y_pred):
    
    # Print accuracy score, precision score, recall score and f1_score for each categories
    for i in range(len(classes)):
        print("Category:", classes[i],"\n", classification_report(y_test.iloc[:, i].values, y_pred[:, i]))
        print('Accuracy of %25s: %.2f' %(classes[i], accuracy_score(y_test.iloc[:, i].values, y_pred[:,i])))


In [17]:
y_test.columns

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [18]:
display_results(y_test, y_pred)

Category: related 
               precision    recall  f1-score   support

           0       0.69      0.41      0.52      1873
           1       0.83      0.94      0.88      5934
           2       0.33      0.38      0.35        58

    accuracy                           0.81      7865
   macro avg       0.62      0.58      0.58      7865
weighted avg       0.79      0.81      0.79      7865

Accuracy of                   related: 0.81
Category: request 
               precision    recall  f1-score   support

           0       0.91      0.98      0.94      6533
           1       0.83      0.51      0.63      1332

    accuracy                           0.90      7865
   macro avg       0.87      0.74      0.79      7865
weighted avg       0.89      0.90      0.89      7865

Accuracy of                   request: 0.90
Category: offer 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      7829
           1       0.00      0.00    

Accuracy of                 transport: 0.96
Category: buildings 
               precision    recall  f1-score   support

           0       0.96      1.00      0.98      7473
           1       0.85      0.13      0.23       392

    accuracy                           0.96      7865
   macro avg       0.90      0.56      0.60      7865
weighted avg       0.95      0.96      0.94      7865

Accuracy of                 buildings: 0.96
Category: electricity 
               precision    recall  f1-score   support

           0       0.98      1.00      0.99      7697
           1       0.88      0.04      0.08       168

    accuracy                           0.98      7865
   macro avg       0.93      0.52      0.53      7865
weighted avg       0.98      0.98      0.97      7865

Accuracy of               electricity: 0.98
Category: tools 
               precision    recall  f1-score   support

           0       0.99      1.00      1.00      7817
           1       0.00      0.00      0.

### 6. Improve your model
Use grid search to find better parameters. 

In [19]:
parameters = {'clf__estimator__max_depth': [10, 50, None],
              'clf__estimator__min_samples_leaf':[2, 5, 10]}

cv = GridSearchCV(pipeline, parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [20]:
cv.fit(X_train.values, y_train.values)
y_pred_2 = cv.predict(X_test)

In [21]:
display_results(y_test, y_pred_2)

Category: related 
               precision    recall  f1-score   support

           0       0.72      0.38      0.50      1873
           1       0.82      0.95      0.88      5934
           2       0.71      0.09      0.15        58

    accuracy                           0.81      7865
   macro avg       0.75      0.47      0.51      7865
weighted avg       0.80      0.81      0.79      7865

Accuracy of                   related: 0.81
Category: request 
               precision    recall  f1-score   support

           0       0.89      0.99      0.93      6533
           1       0.86      0.39      0.53      1332

    accuracy                           0.89      7865
   macro avg       0.87      0.69      0.73      7865
weighted avg       0.88      0.89      0.87      7865

Accuracy of                   request: 0.89
Category: offer 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      7829
           1       0.00      0.00    

Accuracy of               electricity: 0.98
Category: tools 
               precision    recall  f1-score   support

           0       0.99      1.00      1.00      7817
           1       0.00      0.00      0.00        48

    accuracy                           0.99      7865
   macro avg       0.50      0.50      0.50      7865
weighted avg       0.99      0.99      0.99      7865

Accuracy of                     tools: 0.99
Category: hospitals 
               precision    recall  f1-score   support

           0       0.99      1.00      1.00      7787
           1       0.00      0.00      0.00        78

    accuracy                           0.99      7865
   macro avg       0.50      0.50      0.50      7865
weighted avg       0.98      0.99      0.99      7865

Accuracy of                 hospitals: 0.99
Category: shops 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      7837
           1       0.00      0.00      0.00    

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [22]:
pipeline_i = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(DecisionTreeClassifier()))
    ])

In [23]:
pipeline_i.fit(X_train.values, y_train.values)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000002AD4ECE4940>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=DecisionTreeClassifier()))])

In [24]:
y_pred_i = pipeline_i.predict(X_test)

In [25]:
display_results(y_test, y_pred_i)

Category: related 
               precision    recall  f1-score   support

           0       0.51      0.49      0.50      1873
           1       0.85      0.84      0.84      5934
           2       0.16      0.48      0.24        58

    accuracy                           0.75      7865
   macro avg       0.51      0.60      0.53      7865
weighted avg       0.76      0.75      0.76      7865

Accuracy of                   related: 0.75
Category: request 
               precision    recall  f1-score   support

           0       0.90      0.91      0.91      6533
           1       0.55      0.53      0.54      1332

    accuracy                           0.85      7865
   macro avg       0.73      0.72      0.72      7865
weighted avg       0.84      0.85      0.85      7865

Accuracy of                   request: 0.85
Category: offer 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      7829
           1       0.00      0.00    

Accuracy of                     tools: 0.99
Category: hospitals 
               precision    recall  f1-score   support

           0       0.99      0.99      0.99      7787
           1       0.11      0.13      0.12        78

    accuracy                           0.98      7865
   macro avg       0.55      0.56      0.56      7865
weighted avg       0.98      0.98      0.98      7865

Accuracy of                 hospitals: 0.98
Category: shops 
               precision    recall  f1-score   support

           0       1.00      0.99      1.00      7837
           1       0.00      0.00      0.00        28

    accuracy                           0.99      7865
   macro avg       0.50      0.50      0.50      7865
weighted avg       0.99      0.99      0.99      7865

Accuracy of                     shops: 0.99
Category: aid_centers 
               precision    recall  f1-score   support

           0       0.99      0.99      0.99      7762
           1       0.11      0.08      0.

### 9. Export your model as a pickle file

In [26]:
pickle.dump(pipeline_i, open('model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.